In [1]:
from PIL import Image
import glob

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 128*128 사이즈로 바꾸어 저장합니다.
	target_size=(128,128)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

In [2]:
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\scissors"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
가위 이미지 resize 완료!


In [3]:
# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
바위 이미지 resize 완료!


In [4]:
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

0  images to be resized.
0  images resized.
보 이미지 resize 완료!


In [5]:
# !pip install matplotlib
# !pip install scikit-learn
# !pip install scipy

In [6]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

def load_data(img_path, number_of_data=7491):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=128
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.float32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.float32)

    idx=0
    for file in glob.iglob(img_path+'/scissors/*.jpg'):
        img = Image.open(file).resize((img_size, img_size))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.float32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = Image.open(file).resize((img_size, img_size))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.float32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = Image.open(file).resize((img_size, img_size))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.float32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = "dataset" #폴더명
(X, y)=load_data(image_dir_path)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=0, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.4, random_state=0, stratify=y_val)

x_train_norm = X_train/255.0   # 입력은 0~1 사이의 값으로 정규화
x_val_norm = X_val/255.0
print("x_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 7491 입니다.
x_train shape: (4494, 128, 128, 3)
y_train shape: (4494,)


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np

model = models.Sequential()
model.add(layers.Input(shape=(128, 128, 3)))  # 입력 크기 지정
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3, activation='softmax'))
# 😎쌤Tip.
# pretrained model을 써서 가위, 바위, 보 데이터셋을 가지고 fine-tune 파인튜닝 하고 싶어.
# timeframe, temporal modeling
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,392,963 (28.20 MB)

 Trainable params: 7,392,835 (28.20 MB)

 Non-trainable params: 128 (512.00 B)

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

optimizer = keras.optimizers.Adam(learning_rate=0.00001)

model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# ModelCheckpoint 콜백 설정
checkpoint = ModelCheckpoint(
    'dataset/model/best_model006.keras',  # 경로를 지정하여 저장
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)

# # 데이터 증강
# datagen = ImageDataGenerator(
#     rotation_range=20,     # 이미지 회전
#     width_shift_range=0.2, # 가로로 이동
#     height_shift_range=0.2,# 세로로 이동
#     shear_range=0.2,       # 시야각 변환
#     zoom_range=0.2,        # 확대/축소
#     horizontal_flip=True,  # 좌우 반전
#     fill_mode='nearest'
# )

# # 증강된 이미지로 모델 학습
# model.fit(datagen.flow(x_train_norm, y_train),
#             validation_data=(x_val_norm, y_val),
#             epochs=3,
#             callbacks=[checkpoint])   # checkpoint 콜백 추가

# 모델 학습
model.fit(x_train_norm, y_train,
            validation_data=(x_val_norm, y_val),
            epochs=3,
            callbacks=[checkpoint])   # checkpoint 콜백 추가

Epoch 1/3
141/141 ━━━━━━━━━━━━━━━━━━━━ 14s 94ms/step - accuracy: 0.9211 - loss: 1.8424 - val_accuracy: 0.9833 - val_loss: 1.6528
Epoch 2/3
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 94ms/step - accuracy: 0.9457 - loss: 1.7204 - val_accuracy: 0.9889 - val_loss: 1.5840
Epoch 3/3
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 94ms/step - accuracy: 0.9581 - loss: 1.6451 - val_accuracy: 0.9917 - val_loss: 1.5307


In [13]:
import glob

# image_dir_path = "dataset/test"
# (x_test, y_test)=load_data(X_test, X_test.shape[0])
x_test_norm = X_test/255.0

print("x_test shape : {}".format(X_test.shape))
print("y_test shape : {}".format(y_test.shape))

x_test shape : (1199, 128, 128, 3)
y_test shape : (1199,)


In [14]:
from tensorflow import keras
# 저장된 최적의 모델 로드
best_model = keras.models.load_model('dataset/model/best_model006.keras')

# 테스트 데이터로 평가
test_loss, test_accuracy = best_model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

38/38 - 1s - 21ms/step - accuracy: 0.9900 - loss: 1.5370
test_loss : 1.5370079278945923
test_accuracy : 0.9899916648864746


In [15]:
# 테스트 데이터로 평가
test_loss, test_accuracy = best_model.evaluate(x_val_norm, y_val, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

57/57 - 1s - 17ms/step - accuracy: 0.9917 - loss: 1.5307
test_loss : 1.5307351350784302
test_accuracy : 0.9916573762893677


In [16]:
predicted_result = best_model.predict(x_test_norm)	# model이 추론한 확률값
predicted_labels = np.argmax(predicted_result, axis=1)

idx=100		# 값을 변경해서 찾아보자
print('model.predict() 결과 : ', predicted_result[idx])
print('model이 추론한 가장 가능성이 높은 결과 : ', predicted_labels[idx])
print('실제 데이터의 라벨 : ', y_test[idx])

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
model.predict() 결과 :  [9.0153818e-04 3.1251987e-04 9.9878591e-01]
model이 추론한 가장 가능성이 높은 결과 :  2
실제 데이터의 라벨 :  2.0


In [17]:
from sklearn.metrics import classification_report

# 실제 레이블을 y_test에 저장했다고 가정
# 모델 예측
predicted_result = best_model.predict(x_val_norm)
predicted_labels = np.argmax(predicted_result, axis=1)

# 클래스별 f1 score와 accuracy 출력
print(classification_report(y_val, predicted_labels))

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       634
         1.0       1.00      0.98      0.99       566
         2.0       1.00      1.00      1.00       598

    accuracy                           0.99      1798
   macro avg       0.99      0.99      0.99      1798
weighted avg       0.99      0.99      0.99      1798



In [18]:
from sklearn.metrics import classification_report

# 실제 레이블을 y_test에 저장했다고 가정
# 모델 예측
predicted_result = best_model.predict(x_val_norm)
predicted_labels = np.argmax(predicted_result, axis=1)

# 클래스별 f1 score와 accuracy 출력
print(classification_report(y_val, predicted_labels))

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       634
         1.0       1.00      0.98      0.99       566
         2.0       1.00      1.00      1.00       598

    accuracy                           0.99      1798
   macro avg       0.99      0.99      0.99      1798
weighted avg       0.99      0.99      0.99      1798



In [19]:
# import random
# wrong_predict_list=[]
# for i, _ in enumerate(predicted_labels):
#     if predicted_labels[i] != y_test[i]:
#         wrong_predict_list.append(i)
        
# samples = random.choices(population=wrong_predict_list, k=10) # k값을 수정하면 더 많은 값의 수를 볼 수 있다.

# for n in samples:
#     print("예측확률분포: " + str(predicted_result[n]))
#     print("라벨: " + str(y_test[n]) + ", 예측결과: " + str(predicted_labels[n]))
#     # plt.imshow(x_test[n], cmap=plt.cm.binary)
#     print(n)
#     # plt.show()

In [20]:
# model.save('dataset/model/model.keras', include_optimizer=False)